In [1]:
import concurrent.futures
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import time

In [2]:
# generating random 256x256 noise image with error handling
try: 
    noise = np.random.randint(0, 256, (256, 256, 3), dtype = np.uint8)
    image = Image.fromarray(noise)
    image.save("random_noise.jpg")
except Exception as e:
    print(f"Error in generating image: {e}")

In [3]:
# load a pre-trained ResNet model
model = models.resnet18(pretrained=True)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

C:\Users\muSyR\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\muSyR\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def classify_image(image_path):
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(img)
    return output.argmax().item()

image_paths = ["high_res.jpg", "low_res.jpg", "random_noise.jpg"]

In [9]:
# Function to process images using ThreadPoolExecutor
def process_with_threadpool():
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(classify_image, image_paths))
    end_time = time.time()
    total_time = end_time - start_time
    print(f"ThreadPoolExecutor results: {results}")
    print(f"ThreadPoolExecutor total time: {total_time:.4f} seconds")
    return total_time

# Function to process images using ProcessPoolExecutor
def process_with_processpool():
    start_time = time.time()
    try: 
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(executor.map(classify_image, image_paths))
    except Exception as e:
        print(f"Error with Process Pool Execution: {e}")
        results = False
    end_time = time.time()
    total_time = end_time - start_time
    print(f"ProcessPoolExecutor results: {results}")
    print(f"ProcessPoolExecutor total time: {total_time:.4f} seconds")
    return total_time

In [10]:
threadpool_time = process_with_threadpool()
multiprocessing.set_start_method('spawn', force=True)
processpool_time = process_with_processpool()

print(f"\nOverall time comparison:")
print(f"ThreadPoolExecutor total time: {threadpool_time:.4f} seconds")
print(f"ProcessPoolExecutor total time: {processpool_time:.4f} seconds")

ThreadPoolExecutor results: [970, 794, 418]
ThreadPoolExecutor total time: 0.1610 seconds
Error with Process Pool Execution: A process in the process pool was terminated abruptly while the future was running or pending.
ProcessPoolExecutor results: False
ProcessPoolExecutor total time: 0.1721 seconds

Overall time comparison:
ThreadPoolExecutor total time: 0.1610 seconds
ProcessPoolExecutor total time: 0.1721 seconds
